# 生成ネットワーク

リカレントニューラルネットワーク (RNN) とそのゲート付きセルのバリエーション（例えば、長短期記憶セル (LSTM) やゲート付きリカレントユニット (GRU)）は、言語モデル化の仕組みを提供します。つまり、これらは単語の順序を学習し、シーケンス内の次の単語を予測することができます。この特性により、RNNを使って**生成タスク**を実行することが可能になります。例えば、通常のテキスト生成、機械翻訳、さらには画像キャプション生成などです。

前のユニットで議論したRNNアーキテクチャでは、各RNNユニットが次の隠れ状態を出力として生成していました。しかし、各リカレントユニットにもう一つの出力を追加することで、**シーケンス**（元のシーケンスと同じ長さのもの）を出力することが可能になります。さらに、各ステップで入力を受け取らず、初期状態ベクトルだけを受け取り、それに基づいて出力のシーケンスを生成するRNNユニットを使用することもできます。

このノートブックでは、テキスト生成を助けるシンプルな生成モデルに焦点を当てます。簡単のために、**文字レベルのネットワーク**を構築し、文字ごとにテキストを生成します。トレーニング中には、テキストコーパスを取得し、それを文字列のシーケンスに分割する必要があります。


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## 文字ボキャブラリーの構築

文字レベルの生成ネットワークを構築するには、テキストを単語ではなく個々の文字に分割する必要があります。これまで使用していた`TextVectorization`レイヤーではそれができないため、以下の2つの選択肢があります：

* テキストを手動で読み込み、[この公式Kerasの例](https://keras.io/examples/generative/lstm_character_level_text_generation/)のように手作業でトークン化を行う
* 文字レベルのトークン化に`Tokenizer`クラスを使用する

ここでは2つ目の選択肢を選びます。`Tokenizer`は単語単位でのトークン化にも使用できるため、文字レベルから単語レベルのトークン化に簡単に切り替えることが可能です。

文字レベルのトークン化を行うには、`char_level=True`パラメータを渡す必要があります：


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

シーケンスの終了を示す特別なトークンとして**<eos>**を使用したいと考えています。それを語彙に手動で追加しましょう。


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## タイトル生成のための生成型RNNのトレーニング

ニュースタイトルを生成するためにRNNをトレーニングする方法は以下の通りです。各ステップで1つのタイトルを取り出し、それをRNNに入力します。そして、各入力文字に対してネットワークに次の出力文字を生成させます。

![単語 'HELLO' を生成するRNNの例を示す画像。](../../../../../translated_images/ja/rnn-generate.56c54afb52f9781d.webp)

シーケンスの最後の文字に対しては、ネットワークに `<eos>` トークンを生成させます。

ここで使用する生成型RNNの主な違いは、RNNの最終セルだけでなく、各ステップの出力を利用する点です。これを実現するには、RNNセルに `return_sequences` パラメータを指定します。

したがって、トレーニング中のネットワークへの入力は、ある長さのエンコードされた文字列のシーケンスであり、出力は同じ長さのシーケンスですが、1つの要素分シフトされ、最後に `<eos>` で終了します。ミニバッチは複数のそのようなシーケンスで構成され、すべてのシーケンスを揃えるために**パディング**を使用する必要があります。

データセットを変換する関数を作成しましょう。ミニバッチレベルでシーケンスをパディングしたいので、まず `.batch()` を呼び出してデータセットをバッチ化し、その後 `map` を使って変換を行います。この変換関数は、ミニバッチ全体をパラメータとして受け取ります。


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

ここで行ういくつかの重要なこと:
* まず、文字列テンソルから実際のテキストを抽出します
* `text_to_sequences` は文字列のリストを整数テンソルのリストに変換します
* `pad_sequences` はそれらのテンソルを最大長にパディングします
* 最後に、すべての文字をワンホットエンコードし、シフトと `<eos>` の追加も行います。なぜワンホットエンコードされた文字が必要なのかは、すぐにわかるでしょう

ただし、この関数は **Pythonic** であり、つまり Tensorflow の計算グラフに自動的に変換することはできません。この関数を直接 `Dataset.map` 関数で使用しようとするとエラーが発生します。この Pythonic な呼び出しを `py_function` ラッパーを使用して囲む必要があります:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **注**: Python的な変換関数とTensorflowの変換関数を区別するのは少し複雑に感じるかもしれません。また、なぜデータセットを`fit`に渡す前に標準的なPython関数を使って変換しないのか疑問に思うかもしれません。確かにそれも可能ですが、`Dataset.map`を使用することには大きな利点があります。それは、データ変換パイプラインがTensorflowの計算グラフを使用して実行されるため、GPU計算を活用でき、CPUとGPU間でデータをやり取りする必要性が最小限に抑えられる点です。

さて、ジェネレーターネットワークを構築し、トレーニングを開始しましょう。これは、前のユニットで議論した任意の再帰型セル（シンプル、LSTM、またはGRU）に基づくことができます。この例ではLSTMを使用します。

ネットワークは文字を入力として受け取り、語彙サイズが比較的小さいため、埋め込み層は必要ありません。一つのホットエンコードされた入力を直接LSTMセルに渡すことができます。出力層は`Dense`分類器となり、LSTMの出力を一つのホットエンコードされたトークン番号に変換します。

さらに、可変長のシーケンスを扱うため、`Masking`層を使用して、文字列のパディング部分を無視するマスクを作成することができます。これは厳密には必要ではありません。なぜなら、`<eos>`トークン以降の内容にはあまり興味がないからです。しかし、この層タイプの経験を得るために使用してみます。`input_shape`は`(None, vocab_size)`となり、`None`は可変長のシーケンスを示します。また、出力形状も`(None, vocab_size)`となります。以下の`summary`からも確認できます:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## 出力の生成

モデルのトレーニングが完了したので、次はそれを使って出力を生成してみましょう。まず、トークン番号のシーケンスで表現されたテキストをデコードする方法が必要です。このために、`tokenizer.sequences_to_texts` 関数を使うことができますが、文字レベルのトークン化ではうまく機能しません。したがって、トークナイザーから取得したトークンの辞書（`word_index` と呼ばれるもの）を使い、逆マップを作成して独自のデコード関数を作成します。


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

まず、文字列 `start` を用意し、それをシーケンス `inp` にエンコードします。そして、各ステップでネットワークを呼び出し、次の文字を推測します。

ネットワークの出力 `out` は、各トークンの確率を表す `vocab_size` 要素のベクトルです。`argmax` を使用して最も確率の高いトークン番号を見つけ、その文字を生成されたトークンのリストに追加します。そして、生成を続けます。このようにして1文字を生成するプロセスを `size` 回繰り返し、必要な文字数を生成します。`eos_token` が検出された場合は、早期に終了します。


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## トレーニング中の出力サンプリング

*精度*のような有用な指標がないため、モデルが改善しているかどうかを確認する唯一の方法は、トレーニング中に生成された文字列を**サンプリング**することです。そのために、**コールバック**を使用します。これは、`fit`関数に渡すことができ、トレーニング中に定期的に呼び出される関数のことです。


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


この例では、すでにかなり良いテキストを生成していますが、いくつかの方法でさらに改善することができます：

* **テキスト量を増やす**。このタスクではタイトルのみを使用しましたが、全文で試してみるのも良いでしょう。ただし、RNNは長いシーケンスを扱うのが得意ではないため、短い文に分割するか、あらかじめ定義した値 `num_chars`（例えば256）の固定シーケンス長で常にトレーニングするのが理にかなっています。上記の例を変更してそのようなアーキテクチャにするには、[公式のKerasチュートリアル](https://keras.io/examples/generative/lstm_character_level_text_generation/)を参考にすると良いでしょう。

* **多層LSTM**。LSTMセルを2層または3層にするのも試す価値があります。前のユニットで述べたように、LSTMの各層はテキストから特定のパターンを抽出します。文字レベルの生成器の場合、低いLSTM層は音節を抽出し、高い層は単語や単語の組み合わせを抽出すると予想されます。これは、LSTMコンストラクタに層数のパラメータを渡すことで簡単に実装できます。

* **GRUユニット**を試してみて、どちらがより良い結果を出すか比較するのも良いでしょう。また、**異なる隠れ層のサイズ**を試すのも一案です。隠れ層が大きすぎると過学習（例えばネットワークが正確なテキストを学習してしまう）を引き起こす可能性があり、逆に小さすぎると良い結果が得られないかもしれません。


## ソフトテキスト生成と温度

以前の`generate`の定義では、生成されるテキストの次の文字として、常に最も高い確率を持つ文字を選んでいました。この結果、以下の例のように、テキストが同じ文字列の繰り返しになりがちでした。
```
today of the second the company and a second the company ...
```

しかし、次の文字の確率分布を見てみると、いくつかの最も高い確率の間の差がそれほど大きくない場合があります。例えば、ある文字の確率が0.2で、別の文字が0.19である場合などです。たとえば、'*play*'という文字列の次の文字を探すとき、次の文字はスペースでも**e**（単語*player*のように）でも同じくらい適切である可能性があります。

このことから、常に確率が最も高い文字を選ぶのが「公平」とは限らないという結論に至ります。2番目に高い確率の文字を選んでも、意味のあるテキストにつながる可能性があるからです。ネットワークの出力による確率分布から文字を**サンプリング**する方が賢明です。

このサンプリングは、いわゆる**多項分布**を実装する`np.multinomial`関数を使用して行うことができます。この**ソフト**なテキスト生成を実装する関数は以下のように定義されています。


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

私たちは、**温度**と呼ばれるもう1つのパラメーターを導入しました。これは、最高確率にどれだけ厳密に従うべきかを示すために使用されます。温度が1.0の場合、公平な多項分布サンプリングを行い、温度が無限大に近づくとすべての確率が等しくなり、次の文字をランダムに選択します。以下の例では、温度を上げすぎるとテキストが意味をなさなくなり、温度が0に近づくと「循環的な」厳密生成テキストに似てくることが観察できます。



---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてお考えください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は責任を負いません。
